# Packages

In [38]:
import collections

from ortools.sat.python import cp_model

import os
os.getcwd()

'c:\\Users\\gilramolete\\OneDrive - UNIONBANK of the Philippines\\Documents 1\\Route Optimization\\OR-Tools'

# Overview

Companies that manage extensive operations, which require assigning people and resources to tasks at specific times, need to solve difficult scheduling problems on a regular basis. Here are a couple of examples of such problems:
- Schedule employees in **multiple shifts**, subject to a complex set of constraints and staffing requirements.
- Schedule a **manufacturing process that involves performing many tasks on a limited set of machines**, each of which can do only one task at a time.

# Employee Scheduling

Organizations whose employees work multiple shifts need to schedule sufficient workers for each daily shift. Typically, the schedules will have constraints, such as "no employee should work two shifts in a row". Finding a schedule that satisfies all constraints can be computationally difficult.

The following sections present two examples of employee scheduling problems, and show how to solve them using the CP-SAT solver.

## A nurse scheduling problem

In the next example, a hospital supervisor needs to create a schedule for **four nurses over a three-day period**, subject to the following conditions:
- Each day is divided into **three 8-hour shifts**.
- Every day, each shift is assigned to a **single nurse**, and **no nurse works more than one shift**.
- Each nurse is assigned to **at least two shifts during the three-day period**.

The following sections present a solution to the nurse scheduling problem.

In [39]:
# Data
num_nurses = 4
num_shifts = 3
num_days = 3

all_nurses = range(num_nurses)
all_shifts = range(num_shifts)
all_days = range(num_days)

# Create model
model = cp_model.CpModel()

# Create variables
shifts = {}
for n in all_nurses:
    for d in all_days:
        for s in all_shifts:
            shifts[(n, d, s)] = model.NewBoolVar('shift_n%id%is%i' % (n, d, s))

`shifts[(n, d, s)]` equals 1 if shift `s` is assigned to nurse `n` on day `d`, and 0 otherwise.

Next, we show how to assign nurses to shifts subject to the following constraints:
- Each shift is assigned to a single nurse per day.
- Each nurse works at most one shift per day.

Here's the code that creates the first condition.

In [40]:
for d in all_days:
    for s in all_shifts:
        model.AddExactlyOne(shifts[(n, d, s)] for n in all_nurses)

The last line says that for each shift, the sum of the nurses assigned to that shift is 1.

Next, here's the code that requires that each nurse works at most one shift per day.

In [41]:
for n in all_nurses:
    for d in all_days:
        model.AddAtMostOne(shifts[(n, d, s)] for s in all_shifts)

For each nurse, the sum of shifts assigned to that nurse is at most 1 ("at most" because a nurse might have the day off).

Next, we show how to **assign shifts to nurses as evenly as possible**. Since there are nine shifts over the three-day period, we can assign two shifts to each of the four nurses. After that there will be one shift left over, which can be assigned to any nurse.

The following code ensures that each nurse works at least two shifts in the three-day period.

In [42]:
# Try distributing shifts evenly, so that each nurse works min_shifts_per_nurse shifts
# If not possible, because the total number of shifts is not divisible by the number of nurses,
# some nurses will be assigned one more shift
min_shifts_per_nurse = (num_shifts * num_days) // num_nurses

if num_shifts * num_days % num_nurses == 0:
    max_shifts_per_nurse = min_shifts_per_nurse
else:
    max_shifts_per_nurse = min_shifts_per_nurse + 1

for n in all_nurses:
    shifts_worked = []
    for d in all_days:
        for s in all_shifts:
            shifts_worked.append(shifts[(n, d, s)])
    model.Add(min_shifts_per_nurse <= sum(shifts_worked))
    model.Add(sum(shifts_worked) <= max_shifts_per_nurse)

Since there are `num_shifts` * `num_days` total shifts in the schedule period, you can assign at least

```python
(num_shifts * num_days) // num_nurses
```

shifts to each nurse, but some shifts may be left over. (Here, `//` is the Python integer division operator, which returns the floor of the usual quotient.)

For the given values of `num_nurses = 4`, `num_shifts = 3`, and `num_days = 3`, the expression `min_shifts_per_nurse` has the value `(3 * 3 // 4) = 2`, so you can assign at least two shifts to each nurse. This is guaranteed by the constraint:

```python
model.Add(min_shifts_per_nurse <= sum(num_shifts_worked))
```

Since there are nine total shifts over the three-day period, there is one remaining shift after assigning two shifts to each nurse. The extra shift can be assigned to any nurse.

The final line:

```python
model.Add(sum(num_shifts_worked) <= max_shifts_per_nurse)
```

ensures that no nurse is assigned more than one extra shift. The constraint isn't necessary in this case, because there's only one extra shift. But for different parameter values, there could be several extra shifts, in which case the constraint is necessary.

In a non-optimization model, you can enable the search for all solutions.

In [43]:
# Update solver parameters
solver = cp_model.CpSolver()
solver.parameters.linearization_level = 0

# Enumerate all solutions
solver.parameters.enumerate_all_solutions = True

# Register a Solutions Callback
class NursesPartialSolutionPrinter(cp_model.CpSolverSolutionCallback):
    '''Print intermediate solutions.'''

    def __init__(self, shifts, num_nurses, num_days, num_shifts, limit):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._shifts = shifts
        self._num_nurses = num_nurses
        self._num_days = num_days
        self._num_shifts = num_shifts
        self._solution_count = 0
        self._solution_limit = limit
    
    def on_solution_callback(self):
        self._solution_count += 1
        print(f'Solution {self._solution_count}')

        for d in range(self._num_days):
            print('Day %i' % d)
            for n in range(self._num_nurses):
                is_working = False
                for s in range(self._num_shifts):
                    if self.Value(self._shifts[(n, d, s)]):
                        is_working = True
                        print('   Nurse %i works shift %i' % (n, s))
                if not is_working:
                    print('   Nurses {} does not work'.format(n))
        if self._solution_count >= self._solution_limit:
            print('Stop search after %i solutions' % self._solution_limit)
            self.StopSearch()
    
    def solution_count(self):
        return self._solution_count

# Display the first five solutions
solution_limit = 5
solution_printer = NursesPartialSolutionPrinter(shifts, num_nurses, num_days, num_shifts, solution_limit)

# Invoke solver
solver.Solve(model, solution_printer)

# Statistics.
print('\nStatistics')
print('  - conflicts      : %i' % solver.NumConflicts())
print('  - branches       : %i' % solver.NumBranches())
print('  - wall time      : %f s' % solver.WallTime())
print('  - solutions found: %i' % solution_printer.solution_count())

Solution 1
Day 0
   Nurses 0 does not work
   Nurse 1 works shift 0
   Nurse 2 works shift 1
   Nurse 3 works shift 2
Day 1
   Nurse 0 works shift 2
   Nurses 1 does not work
   Nurse 2 works shift 1
   Nurse 3 works shift 0
Day 2
   Nurse 0 works shift 2
   Nurse 1 works shift 1
   Nurse 2 works shift 0
   Nurses 3 does not work
Solution 2
Day 0
   Nurse 0 works shift 0
   Nurses 1 does not work
   Nurse 2 works shift 1
   Nurse 3 works shift 2
Day 1
   Nurses 0 does not work
   Nurse 1 works shift 2
   Nurse 2 works shift 1
   Nurse 3 works shift 0
Day 2
   Nurse 0 works shift 2
   Nurse 1 works shift 1
   Nurse 2 works shift 0
   Nurses 3 does not work
Solution 3
Day 0
   Nurse 0 works shift 0
   Nurses 1 does not work
   Nurse 2 works shift 1
   Nurse 3 works shift 2
Day 1
   Nurse 0 works shift 1
   Nurse 1 works shift 2
   Nurses 2 does not work
   Nurse 3 works shift 0
Day 2
   Nurse 0 works shift 2
   Nurse 1 works shift 1
   Nurse 2 works shift 0
   Nurses 3 does not work
Solu

## Scheduling with shift requests

In this section, we take the previous example and add nurse requests for specific shifts. We then look for a schedule that maximizes the number of requests that are met. For most scheduling problems, it's best to optimize an objective function, as it is usually not practical to print all possible schedules.

This example has the same constraints as the previous example.

In [44]:
# Data
num_nurses = 5
num_shifts = 3
num_days = 7

all_nurses = range(num_nurses)
all_shifts = range(num_shifts)
all_days = range(num_days)

shift_requests = [[[0, 0, 1], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 1],
                   [0, 1, 0], [0, 0, 1]],
                  [[0, 0, 0], [0, 0, 0], [0, 1, 0], [0, 1, 0], [1, 0, 0],
                   [0, 0, 0], [0, 0, 1]],
                  [[0, 1, 0], [0, 1, 0], [0, 0, 0], [1, 0, 0], [0, 0, 0],
                   [0, 1, 0], [0, 0, 0]],
                  [[0, 0, 1], [0, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 0],
                   [1, 0, 0], [0, 0, 0]],
                  [[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 0, 0], [1, 0, 0],
                   [0, 1, 0], [0, 0, 0]]]

# Create model
model = cp_model.CpModel()

In addition to the variables from the previous example, the data also contains a set of triples, corresponding to the three shifts per day. Each element of the triple is 0 or 1, indicating whether a shift was requested. For example, the triple [0, 0, 1] in the fifth position of row 1 indicates that nurse 1 requests shift 3 on day 5.

In [45]:
# Create variables
shifts = {}
for n in all_nurses:
    for d in all_days:
        for s in all_shifts:
            shifts[(n, d, s)] = model.NewBoolVar('shift_n%id%is%i' % (n, d, s))

# Create constraints
for d in all_days:
    for s in all_shifts:
        model.AddExactlyOne(shifts[(n, d, s)] for n in all_nurses)

for n in all_nurses:
    for d in all_days:
        model.AddAtMostOne(shifts[(n, d, s)] for s in all_shifts)

# Try to distribute the shifts evenly, so that each nurse works
# min_shifts_per_nurse shifts. If this is not possible, because the total
# number of shifts is not divisible by the number of nurses, some nurses will
# be assigned one more shift.
min_shifts_per_nurse = (num_shifts * num_days) // num_nurses
if num_shifts * num_days % num_nurses == 0:
    max_shifts_per_nurse = min_shifts_per_nurse
else:
    max_shifts_per_nurse = min_shifts_per_nurse + 1
for n in all_nurses:
    num_shifts_worked = 0
    for d in all_days:
        for s in all_shifts:
            num_shifts_worked += shifts[(n, d, s)]
    model.Add(min_shifts_per_nurse <= num_shifts_worked)
    model.Add(num_shifts_worked <= max_shifts_per_nurse)

# Objective
# pylint: disable=g-complex-comprehension
model.Maximize(sum(shift_requests[n][d][s] * shifts[(n, d, s)] for n in all_nurses 
                for d in all_days for s in all_shifts))
   

Since `shift_requests[n][d][s] * shifts[(n, d, s)` is 1 if shift `s` is assigned to nurse `n` on day `d` and that nurse requested that shift (and 0 otherwise), the objective is the number shift of assignments that meet a request.

In [46]:
# Invoke solver
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Display results
if status == cp_model.OPTIMAL:
    print('Solution:')
    for d in all_days:
        print('Day', d)
        for n in all_nurses:
            for s in all_shifts:
                if solver.Value(shifts[(n, d, s)]) == 1:
                    if shift_requests[n][d][s] == 1:
                        print(f'Nurse {n} works shifts {s} (requested)')
                    else:
                        print(f'Nurse {n} works shift {s} (not requested)')
        print()
    print(f'Number of shift requests met = {solver.ObjectiveValue()}',
        f'(out of {num_nurses * min_shifts_per_nurse})')
else:
    print('No optimal solution found.')

Solution:
Day 0
Nurse 1 works shift 0 (not requested)
Nurse 2 works shifts 1 (requested)
Nurse 3 works shifts 2 (requested)

Day 1
Nurse 1 works shift 0 (not requested)
Nurse 2 works shifts 1 (requested)
Nurse 4 works shifts 2 (requested)

Day 2
Nurse 0 works shift 2 (not requested)
Nurse 3 works shifts 0 (requested)
Nurse 4 works shifts 1 (requested)

Day 3
Nurse 0 works shift 2 (not requested)
Nurse 2 works shifts 0 (requested)
Nurse 3 works shifts 1 (requested)

Day 4
Nurse 0 works shifts 2 (requested)
Nurse 1 works shift 1 (not requested)
Nurse 4 works shifts 0 (requested)

Day 5
Nurse 0 works shift 2 (not requested)
Nurse 3 works shifts 0 (requested)
Nurse 4 works shifts 1 (requested)

Day 6
Nurse 0 works shifts 2 (requested)
Nurse 1 works shift 1 (not requested)
Nurse 2 works shift 0 (not requested)

Number of shift requests met = 13.0 (out of 20)


# The Job Shop Problem

One common scheduling problem is the job shop, in which **multiple jobs are processed on several machines**. Each job consists of a sequence of tasks, which must be performed in a given order, and each task must be processed on a specific machine. For example, the job could be the manufacture of a single consumer item, such as an automobile. The problem is to schedule the tasks on the machines so as to minimize the length of the schedule—the time it takes for all the jobs to be completed.

There are several constraints for the job shop problem:
- No task for a job can be started **until the previous task for that job is completed**.
- A machine can only work on **one task at a time**.
- A task, once started, **must run to completion**.

## Example problem

Below is a simple example of a job shop problem, in which each task is labeled by a pair of numbers (m, p) where *m* is the **number of the machine the task must be processed on** and *p* is the **processing time of the task** — the amount of time it requires. (The numbering of jobs and machines starts at 0.)
- job 0 = [(0, 3), (1, 2), (2, 2)]
- job 1 = [(0, 2), (2, 1), (1, 4)]
- job 2 = [(1, 4), (2, 3)]

In the example, job 0 has three tasks. The first, (0, 3), must be processed on machine 0 in 3 units of time. The second, (1, 2), must be processed on machine 1 in 2 units of time, and so on. Altogether, there are eight tasks.

### A solution

A solution to the job shop problem is an assignment of a start time for each task, which meets the constraints given above. The diagram below shows one possible solution for the problem:

<p align = center>
    <img src = https://developers.google.com/static/optimization/images/scheduling/schedule1.png>
</p>

You can check that the tasks for each job are scheduled at non-overlapping time intervals, in the order given by the problem.

The length of this solution is 12, which is the first time when all three jobs are complete. However, this isn't the optimal solution to the problem.

### Variables and constraints for the problem

This section describes how to set up the variables and constraints for the problem. First, let task(i, j) denote the jth task in the sequence for job i. For example, task(0, 2) denotes the second task for job 0, which corresponds to the pair (1, 2) in the problem description.

Next, define $t_{i, j}$ to be the start time for task(i, j). Thw $t_{i, j}$ are the variables in the job shop problem. Finding a solution involves determining values for these variables that meet the requirement of the problem. There are two types of constraints for the job shop problem:
- *Precendence constraints* -- These arise from the condition that for any two consecutive tasks in the same job, the first must be completed before the second can be started. For example, task(0, 2) and task(0, 3) are consecutive tasks for job 0. Since the processing time for task(0, 2) is 2, the start time for task(0, 3) must be at least 2 units of time after the start time for task 2. (Perhaps task 2 is painting a door, and it takes two hours for the paint to dry.) As a result, you get the following constraint: 
    
    $ t_{0, 2} + \leq t_{0, 3} $
    
- *No overlap constraints* - These arise from the restriction that a machine can't work on two tasks at the same time. For example, task(0, 2) and task(2, 1) are both processed on machine 1. Since their processing times are 2 and 4, respectively, one of the following constraints must hold:

    $ t_{0, 2} + 2 \leq t_{2, 1} $ (if task(0, 2) is scheduled before task(2, 1))

    or

    $ t_{2, 1} + 4 \leq t_{0, 2} $ (if task(2, 1) is scheduled before task(0, 2))

### Objective for the problem

The objective of the job shop problem is to minimize the *makespan*: the length of time from the earliest start time of the jobs to the latest end time.

## A Program solution

In [47]:
# Data
jobs_data = [ # task = (machine_id, processing_time)
    [(0, 3), (1, 2), (2, 2)], # Job 0
    [(0, 2), (2, 1), (1, 4)], # Job 1
    [(1, 4), (2, 3)] # Job 2
]

machines_count = 1 + max(task[0] for job in jobs_data for task in job)
all_machines = range(machines_count)

# Computes horizon dynamically as the sum of all durations
horizon = sum(task[1] for job in jobs_data for task in job)

# Declare model
model = cp_model.CpModel()

# Define variables
# Named tuple to store info about created variables
task_type = collections.namedtuple('task_type', 'start end interval')
# Named tuple to manipulat solution information
assigned_task_type = collections.namedtuple('assigned_task_type', 'start job index duration')

# Creates job intervals and adds to the corresponding machine lists
all_tasks = {}
machine_to_intervals = collections.defaultdict(list)

for job_id, job in enumerate(jobs_data):
    for task_id, task in enumerate(job):
        machine = task[0]
        duration = task[1]
        suffix = '_%i_%i' % (job_id, task_id)
        start_var = model.NewIntVar(0, horizon, 'start' + suffix)
        end_var = model.NewIntVar(0, horizon, 'end' + suffix)
        interval_var = model.NewIntervalVar(start_var, duration, end_var, 'interval' + suffix)
        all_tasks[job_id, task_id] = task_type(start = start_var, end = end_var, interval = interval_var)
        machine_to_intervals[machine].append(interval_var)

For each job and task, the program uses the solver's `NewIntVar` method to create the variables:
- `start_var`: Start time of the task.
- `end_var`: End time of the task.

The upper bound for `start_var` and `end_var` is horizon, the sum of the processing times for all tasks in all jobs. `horizon` is sufficiently large to complete all tasks for the following reason: if you schedule the tasks in non-overlapping time intervals (a non-optimal solution), the total length of the schedule is exactly `horizon`. So the duration of the optimal solution can't be any greater than `horizon`.

Next, the program uses the `NewIntervalVar` method to create an interval variable—whose value is a variable time interval—for the task. The inputs to `NewIntervalVar` are:
- `start_var`: Variable for the start time of the task.
- `duration`: Length of the time interval for the task.
- `end_var`: Variable for the end time of the task.
- `'interval_%i_%i' % (job, task_id))`: Name for the interval variable.

In any solution, `end_var` minus `start_var` must equal duration.

In [48]:
# Define constraints
# Craete and add disjunctive constraints
for machine in all_machines:
    model.AddNoOverlap(machine_to_intervals[machine])

# Precendences inside a job
for job_id, job in enumerate(jobs_data):
    for task_id in range(len(job) - 1):
        model.Add(all_tasks[job_id, task_id + 1].start >= all_tasks[job_id, task_id].end)

The program uses the solver's `AddNoOverlap` method to create the no overlap constraints, which prevent tasks for the same machine from overlapping in time.

Next, the program adds the precedence constraints, which prevent consecutive tasks for the same job from overlapping in time. For each job, the line

```python
model.Add(all_tasks[job, task_id + 1].start >= all_tasks[job, task_id].end)
```

requires the end time of a task to occur before the start time of the next task in the job.

In [49]:
# Makespan objective
obj_var = model.NewIntVar(0, horizon, 'makespan')
model.AddMaxEquality(obj_var, [
    all_tasks[job_id, len(job) - 1].end
    for job_id, job in enumerate(jobs_data)
])
model.Minimize(obj_var)

The expression

```python
   model.AddMaxEquality(
      obj_var,
      [all_tasks[(job, len(jobs_data[job]) - 1)].end for job in all_jobs])
```

creates a variable `obj_var` whose value is the maximum of the end times for all jobs —that is, the makespan.

In [50]:
# Invoke solver
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Display results
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print('Solution:')
    # Create one list of assigned tasks per machine.
    assigned_jobs = collections.defaultdict(list)
    for job_id, job in enumerate(jobs_data):
        for task_id, task in enumerate(job):
            machine = task[0]
            assigned_jobs[machine].append(
                assigned_task_type(start=solver.Value(
                    all_tasks[job_id, task_id].start),
                                   job=job_id,
                                   index=task_id,
                                   duration=task[1]))

    # Create per machine output lines.
    output = ''
    for machine in all_machines:
        # Sort by starting time.
        assigned_jobs[machine].sort()
        sol_line_tasks = 'Machine ' + str(machine) + ': '
        sol_line = '           '

        for assigned_task in assigned_jobs[machine]:
            name = 'job_%i_task_%i' % (assigned_task.job,
                                       assigned_task.index)
            # Add spaces to output to align columns.
            sol_line_tasks += '%-15s' % name

            start = assigned_task.start
            duration = assigned_task.duration
            sol_tmp = '[%i, %i]' % (start, start + duration)
            # Add spaces to output to align columns.
            sol_line += '%-15s' % sol_tmp

        sol_line += '\n'
        sol_line_tasks += '\n'
        output += sol_line_tasks
        output += sol_line

    # Finally print the solution found.
    print(f'Optimal Schedule Length: {solver.ObjectiveValue()}')
    print(output)
else:
    print('No solution found.')

Solution:
Optimal Schedule Length: 11.0
Machine 0: job_1_task_0   job_0_task_0   
           [0, 2]         [2, 5]         
Machine 1: job_2_task_0   job_0_task_1   job_1_task_2   
           [0, 4]         [5, 7]         [7, 11]        
Machine 2: job_1_task_1   job_2_task_1   job_0_task_2   
           [2, 3]         [4, 7]         [7, 9]         

